In [1]:
from tefas import Crawler
import pandas as pd
from datetime import datetime, timedelta

In [2]:
# Define date range for data retrieval
start_date = datetime(2018, 1, 1)
end_date = datetime(2024, 9, 1)
date_increment = timedelta(days=30)

# Initialize the TEFAS Crawler
tefas_crawler = Crawler()

# List of fund codes and columns
fund_codes = ["KPC", "HKH", "KLU", "KZL", "CPU", "KUT"]
columns = ["date", "code", "title", "price", "market_cap", "number_of_investors", "number_of_shares"]

In [3]:
# Data Retrieval Loop
all_funds_data = pd.DataFrame(columns=columns)
for fund in fund_codes:
    current_date = start_date
    while current_date <= end_date:
        try:
            # Fetch data for each fund and date range
            fund_data = tefas_crawler.fetch(
                start=current_date,
                end=current_date + date_increment,
                name=fund
            )
            # Add the fund code to the data
            fund_data['code'] = fund
            # Select only the columns we need
            fund_data = fund_data[['date', 'code', 'title', 'price', 'market_cap', 'number_of_investors', 'number_of_shares']]
            # Drop rows where all elements are NaN
            fund_data = fund_data.dropna(how='all')
            # Check if fund_data is not empty after dropping NaNs
            if not fund_data.empty:
                all_funds_data = pd.concat([all_funds_data, fund_data], ignore_index=True)
        except Exception as e:
            print(f"Error fetching data for {fund} from {current_date}: {e}")
        # Move to the next date range
        current_date += date_increment

C:\Users\furka\AppData\Local\Temp\ipykernel_19940\1536545443.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_funds_data = pd.concat([all_funds_data, fund_data], ignore_index=True)


In [4]:
# Sort the DataFrame by 'date' column
all_funds_data = all_funds_data.sort_values(by='date')
all_funds_data = all_funds_data.reset_index(drop=True)

In [5]:
all_funds_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4707 entries, 0 to 4706
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   date                 4707 non-null   object 
 1   code                 4707 non-null   object 
 2   title                4707 non-null   object 
 3   price                4707 non-null   float64
 4   market_cap           4707 non-null   float64
 5   number_of_investors  4707 non-null   float64
 6   number_of_shares     4707 non-null   float64
dtypes: float64(4), object(3)
memory usage: 257.5+ KB


In [6]:
all_funds_data

,date,code,title,price,market_cap,number_of_investors,number_of_shares
0,2019-09-23,KZL,KUVEYT TÜRK ASSET MANAGEMENT GOLD PARTICIPATIO...,1.175174,8.308115e+06,298.0,7.069690e+06
1,2019-09-24,KZL,KUVEYT TÜRK ASSET MANAGEMENT GOLD PARTICIPATIO...,1.193278,8.495458e+06,295.0,7.119429e+06
2,2019-09-25,KZL,KUVEYT TÜRK ASSET MANAGEMENT GOLD PARTICIPATIO...,1.185895,8.384649e+06,288.0,7.070313e+06
3,2019-09-26,KZL,KUVEYT TÜRK ASSET MANAGEMENT GOLD PARTICIPATIO...,1.194965,8.071311e+06,287.0,6.754434e+06
4,2019-09-27,KZL,KUVEYT TÜRK ASSET MANAGEMENT GOLD PARTICIPATIO...,1.181067,7.889600e+06,285.0,6.680062e+06
...,...,...,...,...,...,...,...
4702,2024-08-29,HKH,HEDEF PORTFÖY KATILIM HİSSE SENEDİ (TL) FONU (...,5.953492,2.550287e+09,34485.0,4.283682e+08
4703,2024-08-29,KZL,KUVEYT TÜRK ASSET MANAGEMENT GOLD PARTICIPATIO...,11.463023,2.102733e+10,40340.0,1.834362e+09
4704,2024-08-29,KPC,KUVEYT TÜRK PORTFÖY KATILIM HİSSE SENEDİ (TL) ...,11.523798,4.624796e+09,34284.0,4.013256e+08
4705,2024-08-29,KUT,KUVEYT TÜRK PORTFÖY KIYMETLİ MADENLER KATILIM ...,4.603914,1.123731e+09,13807.0,2.440818e+08


In [7]:
all_funds_data.to_csv('tefas_funds_data.csv', index=False, encoding='utf-8')